### Cats and Dogs Classification using CNN

In this workshop, I'm going to create CNN by my own.
We will use pytorch.

In [ ]:
import torch 
import torch.nn as nn 
import torch.nn.functional as F 
import datasets
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder 
from torch.utils.data import DataLoader

In [ ]:
torch.__version__

In [ ]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

In [ ]:
train_path = "../pytorch/CatDogDataset/train"
test_path = "../pytorch/CatDogDataset/test"
train_dataset = ImageFolder(root=train_path, transform=transform)
test_dataset = ImageFolder(root=test_path, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)

In [ ]:
train_dataset

In [ ]:
test_dataset

### Craete Fully Connected Network

In [ ]:
class CNN(nn.Module):
    def __init__(self, in_channel=3, num_classes=2):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=in_channel, out_channels=8, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.pool = nn.MaxPool2d(kernel_size=(3,3), stride=(1,1))
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.fc1 = nn.Linear(16*63504, num_classes)  # 128/2 = 64 -> 64/2 = 32        15376
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
        
        return x

In [ ]:
model = CNN()

### Set Device

In [ ]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
device

### Hyperparameters

In [ ]:
learning_rate = 0.001 
batch_size = 8 
num_epochs = 15

### Initialize Network

In [ ]:
model = CNN().to(device)

### Loss and Optimizer

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

### Train

In [ ]:
for epoch in range(num_epochs):
    print("Epoch", epoch)
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to CUDA if possible
        data = data.to(device)
        targets = targets.to(device)
        
        # Score
        scores = model(data)
        loss = criterion(scores, targets)
        
        # Backward
        optimizer.zero_grad()
        loss.backward()
        
        # Gradient Descent or Adam Step
        optimizer.step() # Update Weight
        
        print(f"Loss : {loss}")

In [ ]:
train_dataset[2][0]

In [ ]:
train_dataset.targets

### Evaluation

In [ ]:
def check_accuracy(loader, model):
    if train_loader:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")  
          
    num_correct = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            # x = x.reshape(x.shape[0], -1)
            
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
            
        print(f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}")
        
    model.train()

In [ ]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

In [ ]:
import matplotlib.pyplot as plt 

model.eval()

data, target = test_dataset[133]

data = data.unsqueeze(0).to(device)

output = model(data)

prediction = output.argmax(dim=1, keepdim=True).item()

print(f'Prediction : {prediction}')

classes = ['Cat', 'Dog']
print("Predicted : ", classes[prediction])

image = data.squeeze(0).cpu().numpy().transpose(1, 2, 0)  # Transpose to (28, 28, 3)

plt.imshow(image, cmap='gray')
plt.show()


In [ ]:
test_dataset[0]

### Save Model

In [ ]:
import pickle

filename = "catdogclassification.sav"
pickle.dump(model, open(filename, 'wb'))

### Load Model

In [ ]:
import pickle

filename = "catdogclassification.sav"
loaded_model = pickle.load(open(filename, 'rb'))

In [ ]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

import torchvision
from PIL import Image

input_path = r"../pytorch/cat_test.jpg"
input_data = Image.open(input_path)  # Open image with PIL
# input_data = torchvision.io.read_image(path=input_path)
input_data = transform(input_data)
input_data = input_data.squeeze(0).cpu().numpy().transpose(1, 2, 0)  # Transpose to (28, 28, 3)

plt.imshow(input_data)
plt.show()

In [ ]:
input_data = torch.tensor(input_data)
# expected 3 128 128
input_data = input_data.reshape(3,256,256)

In [ ]:
print(input_data)
print(input_data.shape)

In [ ]:
model.eval()

data = input_data.unsqueeze(0)

data = torch.tensor(data,dtype=torch.float)

output = model(data)

prediction = output.argmax(dim=1, keepdim=True).item()

print(f'Prediction : {prediction}')

classes = ['Cat', 'Dog']
print("Predicted : ", classes[prediction])

In [ ]:
data = torch.tensor(data)
data

In [34]:
import os
import numpy as np
import json
import pandas as pd
import torch.nn.functional as F
import re
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.backends.cudnn as cudnn
import random

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
np.random.seed(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)

In [35]:
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')
device

device(type='cuda')

In [36]:
device

device(type='cuda')